In [1]:
import time
import json
import datetime
import pandas as pd 
import json
import os,sys
import numpy as np
import csv
from pandas.core.frame import DataFrame
import copy
import matplotlib.pyplot as plt
import nltk
import util
# from util import change_year_type, group_different, build_dic_function,build_group_dic,judge_success
import networkx as nx

In [2]:
#"CompanyID","CompanyName","YearFounded"
com1 = pd.read_excel('Company.xlsx')  
com1.reset_index(drop=True, inplace=True)
#"CompanyID","IndustrySector","IndustryGroup","IndustryCode","IsPrimary"
com_industry = pd.read_excel('CompanyIndustryRelation.xlsx')
com_industryfouh.reset_index(drop=True, inplace=True)
#"CompanyID","PersonID	PersonName	FullTitle	IsCurrent	StartDate	EndDate"
com_team = pd.read_excel('CompanyTeamRelation.xlsx')
com_team.reset_index(drop=True, inplace=True)
#CompanyID	LocationName	LocationType	LocationStatus	Address1	Address2	City	State	PostCode	Country	OfficePhone	OfficeFax
com_location = pd.read_excel('CompanyLocationRelation.xlsx')
com_location.reset_index(drop=True, inplace=True)

#PersonID	DealID	CompanyID	CompanyName	DealDate	DealType	DealSize
Deal = pd.read_excel('PeopleAffiliatedDealRelation.xlsx')
Deal.reset_index(drop=True, inplace=True)
#PersonID	EntityID	EntityName	EntityType	FullTitle	IsCurrent	IsBoardSeat	Location	StartDate	EndDate	LastUpdated
peo_pos = pd.read_excel('PeoplePositionRelation.xlsx')  
peo_pos.reset_index(drop=True, inplace=True)

# 构建网络 & 计算中心度

In [3]:
foundyear_dic = util.build_dic_function(list(com1["CompanyID"]),list(com1["YearFounded"]),"None")  ####### company:foundyear

In [4]:
# 找到每年之前，每个人对应的待过/现任公司列表
yy = []
for i in range(1980,2020):
    yy.append(i)

def build_network():
    dic = {}
    whether_company_team = {} #作为teammember最早的年份
    for year in yy:
#         print(year)
        dic[year] = {}
        for i in range(len(com_team)):
            ss = util.change_year_type(com_team["StartDate"][i])
            if (pd.isnull(ss)) or (ss > year):
                continue
            pp = com_team["PersonID"][i]
            cc = com_team["CompanyID"][i]
            if pp not in whether_company_team.keys():
                whether_company_team[pp] = ss
            else:
                whether_company_team[pp] = min(whether_company_team[pp],ss)
            if pp not in dic[year].keys():
                dic[year][pp] = []
            if cc not in dic[year][pp]:
                dic[year][pp].append(cc)
        for i in range(len(peo_pos)):
            ss = util.change_year_type(peo_pos["StartDate"][i])
            if (pd.isnull(ss)) or (ss > year):
                continue
            pp = peo_pos["PersonID"][i]
            cc = peo_pos["EntityID"][i]
            if pp not in dic[year].keys():
                dic[year][pp] = []
            if cc not in dic[year][pp]:
                dic[year][pp].append(cc)
    ################ print part###################
    count = 0
    for test_year in yy:
        for pp in dic[test_year]:
            if len(dic[test_year][pp])>1:
                count += 1
#         print(test_year,count)
    print("-------------------- stage 1 -------------------------")
    ##############################################
    return dic,whether_company_team
    
dic,whether_company_team = build_network()

-------------------- stage 1 -------------------------


In [ ]:
print(dic[2010]["207305-83P"],dic[2015]["207305-83P"])

In [7]:
print(dic[2010]["207305-83P"],dic[2015]["207305-83P"])

['268701-40'] ['268701-40']


In [6]:
print(whether_company_team["86948-74P"])

2014


In [7]:
chuangye = list(com1["CompanyID"])
def calculate_edge2(list_):
    yes = []
    no = []
    for tmp in list_:
        if (tmp in chuangye):
            yes.append(tmp)
        else:
            no.append(tmp)
    output = []
    for yes_com in yes:
        for no_com in no:
            tmp_out = sorted([yes_com,no_com])
            output.append((tmp_out[0],tmp_out[1]))
    return output

print(calculate_edge2(dic[2019]["39837-97P"]))
#[('150687-46', '53612-02'), ('433199-80', '53612-02')]

[('150687-46', '53612-02'), ('433199-80', '53612-02')]


In [9]:
for tmp in dic[2019]["39837-97P"]:
    print(tmp in chuangye)

True
True
False


In [8]:
# 对每年 每个人所处公司列表，两两连接
def stage_two(dic,whether_company_team):
    net_dic = {}
    # 对每年 每个人所处公司列表，两两连接
    for year in yy:
        net_dic[year] = []
        for pp in dic[year].keys():
            if (pp not in whether_company_team.keys()) or whether_company_team[pp] > year:
                continue
            list_ = sorted(dic[year][pp]) #按公司ID排好序的列表
            if len(list_) == 1:
                continue
            net_dic[year] = net_dic[year] + calculate_edge2(list_)
#             for ii in range(len(list_)):
#                 for jj in range(ii,len(list_)):
#                     if ii == jj:
#                         continue
#                     net_dic[year].append((list_[ii],list_[jj]))
        net_dic[year] = list(set(net_dic[year]))
    return net_dic
net_dic = stage_two(dic,whether_company_team)

In [12]:
print(len(net_dic[2019]))

45644


In [28]:
print(len(net_dic[2012]))

13085


In [18]:
print(len(net_dic[2012]))

26930


In [9]:
def save_file (add_dic):
    add_dic = [["year","company","value"]] + add_dic
    add_dic = pd.DataFrame(add_dic)
    add_dic.to_excel('closeness_centrality_company_major_9.8.xlsx', header=False, index=False)

index_save = [["年份","边总数","节点总数","最大连通分支包含的节点个数","最大连通分支比例"]]
def stage_three(net_dic):
    # 计算网络中心度
    output_list = []
    yy = []
    for i in range(2010,2016):
        yy.append(i)
    for year in yy:
        node = []
        kkkk = []
        G = nx.Graph()
        for pair in net_dic[year]:
            if pair[0] not in node:
                node.append(pair[0])
            if pair[1] not in node:
                node.append(pair[1])
            G.add_edge(pair[0],pair[1])
        matrix = node
        center = nx.closeness_centrality(G)
        max_cc =max(nx.algorithms.components.connected_components(G),key = len)
        print('--------- ',year,' ------------')
        kkkk.append(max_cc)
#         print(max_cc)
        print("该网络的节点总数为：",len(matrix))
        ncc = float(len(max_cc))
        print("最大连通分支包含的节点个数：",ncc)
        print("最大连通分支比例：",ncc/len(matrix))
        for item in center.items():
            if pd.isnull(item[0]) or pd.isnull(item[1]):
                continue
            output_list.append([year,item[0], str(item[1])])
        index_save.append([year,len(net_dic[year]),len(matrix),ncc,ncc/len(matrix)])
    save_file(output_list)
    return kkkk
max_node = stage_three(net_dic)

---------  2010  ------------
该网络的节点总数为： 8229
最大连通分支包含的节点个数： 784.0
最大连通分支比例： 0.09527281565196258
---------  2011  ------------
该网络的节点总数为： 11092
最大连通分支包含的节点个数： 1478.0
最大连通分支比例： 0.13324918860439958
---------  2012  ------------
该网络的节点总数为： 14993
最大连通分支包含的节点个数： 2610.0
最大连通分支比例： 0.17408123791102514
---------  2013  ------------
该网络的节点总数为： 19829
最大连通分支包含的节点个数： 4301.0
最大连通分支比例： 0.21690453376367946
---------  2014  ------------
该网络的节点总数为： 25563
最大连通分支包含的节点个数： 6738.0
最大连通分支比例： 0.26358408637483866
---------  2015  ------------
该网络的节点总数为： 31376
最大连通分支包含的节点个数： 9645.0
最大连通分支比例： 0.30740056093829676


In [10]:
out = pd.DataFrame(index_save)
out.to_excel("tmp.xlsx",header = False,index=False)

# 计算中心度与成功率的关系

In [4]:
#"IPO","Merger of Equals","Merger/Acquisition","Reverse Merger"
def build_ipoma(list1, list2,time):
    # list1:list2
    yy = 7 #限制未来7年
    dic = {}
    dic_time = {}
    kkk = []
    for i in range(len(list1)):
        item1,item2 = list1[i],list2[i]
        if item2 not in ["Bankruptcy: Admin/Reorg","Bankruptcy: Liquidation","Dividend","Dividend Recapitalization","IPO","Merger of Equals","Merger/Acquisition","Reverse Merger","Share Repurchase"]:
            continue
        if pd.isnull(item1) or pd.isnull(item2):
            continue
        try:
            ff = foundyear_dic[item1]
            dd = util.change_year_type(time[i])
            if (dd == None):
                continue
            #取自成立后五年内的最后一个结果
            if item1 not in dic.keys():
                if (dd-ff) <= yy:
                    dic[item1] = item2
                else:
                    dic[item1] = "More5"
                dic_time[item1] = dd
            else:
                if (dd-ff) > yy:
                    continue
                dd2 = dic_time[item1]
                if dd > dd2:
                    dic[item1] = item2
                    dic_time[item1] = dd
        except KeyError:
            continue
    return dic,dic_time

ipoma_dic,ipoma_time_dic = build_ipoma(list(Deal["CompanyID"]),list(Deal["DealType"]),list(Deal["DealDate"]))  ####### company:outcome

def append_what(com):
    if com in ipoma_dic.keys():
        string = ipoma_dic[com]
        if string in ["IPO","Merger of Equals","Merger/Acquisition","Reverse Merger"]:
            return 1
        else:
            return 0
    else:
        return 0

In [12]:
close = pd.read_excel('Table/network/closeness_centrality_company_major.xlsx')
# close = pd.read_excel('Table/network/closeness_centrality_pp_cooperation.xlsx')
close.reset_index(drop=True, inplace=True)

In [14]:
yy = []
for i in range(1980,2016):
    yy.append(i)
def build_dic_1(pd):
    dic = {}
    dic_cal = {}
    for year in yy:
        dic[year] = {}
        dic_cal[year] = []
    Y = list(pd["year"])
    P = list(pd["company"])
    V = list(pd["value"])
    for i in range(len(Y)):
        dic[Y[i]][P[i]] = V[i]
        dic_cal[Y[i]].append(V[i])
    return dic,dic_cal
close_dic,sum_close_dic = build_dic_1(close)

In [15]:
#计算前百分之及的成功率
yy = []
for i in range(2000,2016):
    yy.append(i)

for year in yy:
#     kk = np.percentile(sum_close_dic[year],0.8)
#     print(kk)
    nn = int(len(sum_close_dic[year])*0.2)
    diff_value = sorted(sum_close_dic[year])
    kk = diff_value[nn]
    calc = []
    for company in close_dic[year].keys():
        #在该年之前上市
        if (company in ipoma_time_dic.keys()) and (ipoma_time_dic[company] < year):
            continue
        if close_dic[year][company] <= kk:
            calc.append(append_what(company))
#     print(year)
#     print(len(calc))
    print(sum(calc)/len(calc))
#     print(calc)

0.002742230347349177
0.0030349013657056147
0.0025823111684958036
0.0033500837520938024
0.0023832221163012394
0.0039447731755424065
0.0043859649122807015
0.008559201141226819
0.010565110565110565
0.01237675687014894
0.013712718546451834
0.015064562410329985
0.01430615164520744
0.014990377798035045
0.013503437991881369
0.011673151750972763


In [11]:
#计算top多少多少的成功率
for year in yy:
#     nn = int(len(sum_close_dic[year])*0.2)
    diff_value = sorted(close_dic[year].items(),key=lambda k: k[1],reverse=True)
    count = 0
    calc = []
    for kkk in diff_value:
        if count == 800:
            break
        count += 1
        company = kkk[0]
        if (company in ipoma_time_dic.keys()) and (ipoma_time_dic[company] < year):
            continue
        calc.append(append_what(company))
#     print(len(calc))
    print(sum(calc)/len(calc))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
